# TD1 MANIPULATIONS BASIQUES

### Exercice 1.1

In [ ]:
#TD1 : 
#Exercice 1 et 3 dans les cadres du notebook
#Exercices 2 dans Rstudio

#########################
#########################
#EXERCICE 1: manipuler des objets (scalaires, strings, vecteurs, dataframes)

#cadre 1: scalaires
###################
a<-4
b<-18
c<-45
d<-46

#age total:
tot<-a+b+c+d
tot
#moyenne:
moy<-tot/4
moy
#diff
d-a

In [ ]:
#cadre 2: vecteurs
##################
v<-c(a,b,c,d)

#age total
tot<-sum(v)
tot
#moyenne:
moy<-mean(v)
moy
#diff
v[4]-v[1]

#dans 10 ans:
v10<-v+10
v10

In [ ]:
#cadre 3: data
##############

#dataframe
df_famille <- data.frame(NOM=c("a","b","c","d"),
                         AGE=v,
                         GENRE = c("f","g","f","g"))
df_famille

#nature des objets et taille
class(df_famille)
dim(df_famille)

class(v)
length(v)

In [ ]:
#cadre 4: data (suite)
######################
#année d'anniversaire des 100 ans
df_famille$naissance <- 2019-v
df_famille$centans <-df_famille$naissance+100
df_famille

#moyenne d'âge des femmes: 
mean_f <- (df_famille[1,2] + df_famille[3,2])/2
mean_f

### Exercice 1.2 (dans RSTUDIO)

In [ ]:
#########################
#########################
#EXERCICE 2: définir un répertoire et ouvrir des données

#setwd("C:/Users/lmartinoty/Dropbox/Paris1/Econométrie/M1Finance_20192020/Material_EPI/TD1")
#attention : /, pas \ ! ne pas oublier les guillemets. 

world <-read.csv("TD1_world_small.csv")

### Exercice 1.3

In [ ]:
#########################
#########################
#EXERCICE 3: installer des progiciels (packages)

install.packages("plyr") #installation
install.packages("dplyr") 
install.packages("ggplot2")
library(plyr) #chargement
library(dplyr) 
library(ggplot2)

# TD2 MANIPULATIONS DES DONNEES

### Code prérequis

In [ ]:
library(plyr)
library(dplyr)

In [ ]:
#TD2 : 
#Exercice uniquement dans les cadres du notebook

#########################
#########################
#EXERCICE : manipuler des données

#cadre 1: ouvrir les données
###################

world <-read.csv("TD1_world_small.csv")
head(world)

In [ ]:
#cadre 2: création des 2 variables
###################
summary(world)

# polityIV2
world$polityIV2<-world$polityIV-10

#rich/poor
cutoff<-mean(world$gdppcap08)
cutoff
world$rich<-ifelse(world$gdppcap08>cutoff,1,0)

head(world)

In [ ]:
#cadre 3: sous échantillon des pays européens
###################
levels(world$region)
europe<-filter(world,region=="C&E Europe"|region=="W. Europe"|region=="Scandinavia")
summary(europe)

In [ ]:
#cadre 4: suppression de la variable région
###################

europe<-select(europe,-region)
head(europe)

In [ ]:
#cadre 5: ordonner les données selon polityIV2
###################

europe<-arrange(europe,polityIV2)
head(europe)

In [ ]:
#cadre 6: piping it all
###################
world<-read.csv("TD1_world_small.csv")

europe <- world %>%
  mutate(polityIV2=polityIV-10,
         rich=ifelse(gdppcap08>mean(gdppcap08),1,0)) %>%
  filter(region=="C&E Europe"|region=="W. Europe"|region=="Scandinavia") %>%
  select(-region) %>%
  arrange(polityIV)
europe

In [ ]:
#cadre 7: petites questions
###################

table(europe$rich) #27 pays riches et 14 pauvres si on prend comme base la moyenne du PIB par habitant

europe$democ<-ifelse(europe$polityIV>=18,1,0)
mean(europe$democ) #68% de scores polity de 18 et + en Europe.

# TD 3 STATISTIQUES DESCRIPTIVES

### Code prérequis

In [ ]:
#NOTE ces lignes sont seulement pour Rstusio, pas pour le notebook:
library(dplyr)
library(foreign)
library(plyr)
library(dplyr)

#########################
#########################
#On ouvre les données
anes <- read.dta("TD3_anescum_small_v12.dta")

#On sélectionne et renomme quelques variables
anes <- anes %>% 
  select(year = VCF0004,
         age = VCF0101,
         gender = VCF0104,
         race = VCF0106,
         edu = VCF0140,
         south = VCF0113,
         income = VCF0114,
         partyid = VCF0301,
         interest = VCF0310,
         govtrust = VCF0604,
         abortion = VCF0838,
         demtherm = VCF0218)

#on regarde les 6 premières observations:
head(anes)

#########################
#########################
#CREATION DE DONNEES

#opinion politique des américains recodée en 3 catégories:
anes$partyid3 <- anes$partyid #dupliquer
levels(anes$partyid3)<-c(NA,
                         "Democrat","Democrat","Democrat",
                         "Independent",
                         "Republican","Republican","Republican") #redéfinir catégories
table(anes$partyid3) #nombre
prop.table(table(anes$partyid3)) #proportion

#on simplifie encore l'opinion: démocrates contre le reste (indépendant+épublicains)
anes$democrat <- anes$partyid3
levels(anes$democrat) <- c(1, 0, 0)

#on crée la variable mean qui est la proportion de démocrates dna schacune des régions pour chaque année
anes$democrat <- as.numeric(as.character(anes$democrat)) #on transforme la variable catégorielle en variable numérique
ddply(anes, .(gender), summarise,
      mean = mean(democrat, na.rm = T),
      median = median(democrat, na.rm = T))

### Exercice 3.1

In [ ]:
#EXERCICE 1 : 

#Cadre 1
####
#Etudier la distribution de la variable *interest* (une variable catégorielle qui mesure 'intérêt porté à la politique), c'est-à-dire : 
#- le nombre 
#- et la proportion d'observations pour chaque catégorie de la variable.

class(anes$interest)
levels(anes$interest)<-c(NA,"Not much","Somewhat","Very",NA)
table(anes$interest, exclude=NULL)
t<-prop.table(table(anes$interest))
t
barplot(t)

In [ ]:
#Cadre 2
####
#Trouver les statistiques descriptives d'une variable numérique de votre choix: moyenne, médiane, écart type, minimum, maximum, 7 et 45ème percentile.
head(anes)
class(anes)
class(anes$govtrust)
class(anes$age) #celle-ci est numérique

summary(anes$age)
sd(anes$age, na.rm=T)
quantile(anes$age, probs=c(0.07,0.45), na.rm=T)

### Exercice 3.2

In [ ]:
#########################
#########################
#EXERCICE 2 : 

#Cadre 1
####
#Créer un tableau croisé entre *south* (Sud/Nord) et la variable *partyid3* (parti démocrate, répubicain ou indépendant), séparément pour 1952 et 2008 (variable *year*). Analyser l'évolution.

#revoir les données: 
class(anes$south)
levels(anes$south)
class(anes$partyid3)
levels(anes$partyid3)

#sous-échantillon pour 1952 et 2008
anes1952 <- filter(anes, year==1952)
anes2008 <- filter(anes, year==2008)

#fréquences
t1952<-table(anes1952$partyid3,anes1952$south)
t2008<-table(anes2008$partyid3,anes2008$south)

#proportions, avec la bonne valeur de margin (somme à 100 dans le sud et dans le nord)
prop.table(t1952, margin=2)
prop.table(t2008, margin=2)

#réponse: 
#le sud est devenu davantage républicain (de 15 à 30%), 
#et le nord est devenu de plus en plus démocrate (52 à 62%).
# partout les indépendants progressent. 

In [ ]:
#Cadre 2
####
#Utiliser la variable *democrat* crée ci-dessus et la fonction **ddply()** pour trouver la proportion de démocrates, et l'écart type de *democrat*, par région (sud/non-sud) et année (toutes les valeurs, pas seulement 1952 et 2008).  
ddply(anes, .(south,year), summarise, mean=mean(democrat, na.rm=T), sd= sd(democrat, na.rm=T))

### Exercice 3.3

In [ ]:
#########################
#########################
#EXERCICE 3 : 


#Cadre 1
####
#Représenter le nuage de points (scatter plot) du thermomètre démocrate (*demtherm*) en fonction de l'âge (*age*) pour 1978. 

anes_sub <- subset(anes, year %in% c(1978))
ggplot(anes_sub, aes(x=age,y=demtherm)) +
  geom_point(size=0.5)

In [ ]:
#Cadre 2
####
#Refaire le même nuage de points pour les années 1978, 1886, 1994 et 1998. Utiliser **facet_wrap()**.

anes_sub <- subset(anes, year %in% c(1978, 1986, 1994, 1998))
ggplot(anes_sub, aes(x=age,y=demtherm)) +
  geom_point(size=0.3) +
  facet_wrap(~year)

In [ ]:
#Cadre 3
####
#Dans l'exercice 2 nous avons calculé la proportion de démocrates par région (*south*) et année (*year*). Utiliser **ggplot** pour représenter ces estimations. Le temps est en abscisse, l'appartenance au parti démocrate en ordonnée. Relier les points par des lignes. Conclure. 

t<-ddply(anes,.(south,year), summarize,mean=mean(demtherm, na.rm=T))
head(t)
ggplot(t, aes(x=year,y=mean, color=south)) +
  geom_point() +
  geom_line()

#Note: pour éviter les variables manquantes, 
#cette commande crée t2 à partir de t en éliminant toutes les lignes d'observation pour lesquelles la variable mean est manquante (NA)
t2 <- t[!is.na(t$mean), ]
#on refait ensuite le même graphique, avec les données t2:
ggplot(t2, aes(x=year,y=mean, color=south)) +
  geom_point() +
  geom_line()

# TD4 MOINDRES CARRES ORDINAIRES

### Code Prérequis 

In [ ]:
#Exercice uniquement dans les cadres du notebook

#NOTE ces lignes sont seulement pour Rstusio, pas pour le notebook:
library(dplyr)
library(foreign)
library(plyr)
library(dplyr)

#########################
#########################
#On ouvre les donnÃ©es
anes <- read.dta("TD3_anescum_small_v12.dta")

#On sÃ©lectionne et renomme quelques variables
anes <- anes %>% 
  select(year = VCF0004,
         age = VCF0101,
         gender = VCF0104,
         race = VCF0106,
         edu = VCF0140,
         south = VCF0113,
         income = VCF0114,
         partyid = VCF0301,
         interest = VCF0310,
         govtrust = VCF0604,
         abortion = VCF0838,
         demtherm = VCF0218)

#on regarde les 6 premiÃ¨res observations:
head(anes)

#########################
#########################
#CREATION DE DONNEES

#valeurs manquantes: on recode gender
levels(anes$gender)
levels(anes$gender) <- c(NA, "Homme", "Femme")

#correction de age
anes$age[anes$age==0]<-NA
sort(unique(anes$age))

#correction des quintiles
levels(anes$income)[1]<-NA
levels(anes$income)

saveRDS(anes, file="TD4_mydata.rds")

#########################
#########################
#REGRESSION

lm2 <- lm(demtherm~gender+income+age, data=anes)

### Exercice 4.1

In [ ]:
#########################
#########################
#EXERCICE 1 : Réaliser le graphique type 'Option 1' qui montre l'effet du genre sur le thermomètre démocrate, avec le modèle lm2.

#voir les objets
coef(lm2)
confint(lm2,level=0.95)

#sélection des objets pertinents
est2<-coef(lm2)[2]
c2<-confint(lm2,level=0.95)[2,]

#création d'un dataframe
est2 <- data.frame(est = est2,
                   c_inf = c2[1],
                   c_sup = c2[2], 
                   model = "ModÃ¨le multiple")

#voir le dataframe crée:
est2

#graphique
ggplot(est2,aes(x=model,y=est)) +
  geom_point() +
  geom_errorbar(aes(ymin=c_inf,ymax=c_sup,width=0.1)) +
  geom_hline(yintercept=0, color="white") +
  xlab("") +
  ylab("Opinion des femmes sur le parti dÃ©mocrate (relativement aux hommes)")

### Exercice 4.2

In [ ]:
#########################
#########################
#EXERCICE 2 : 

#Cadre 1
####
#On souhaite connaître l'impact de l'ethnie sur le score pro-démocrate. 
#On estime un modèle où le score demtherm dépend: du genre gender, de l'âge age, du niveau d'éducation edu, de l'ethnie race. 
#Le codebook nous apprend que race prend 3 valeurs: 1 si 'white', 2 si 'black', 3 si 'other', 9 si 'missing'.
#Commencez par restaurer l'objet "TD4_mydata.rds". Créez un dataframequi contient uniquement les données qui nous intéressent (rappel: TD2: **library(dplyr)**, piping **%>%**, fonction **select()**).

library(plyr)
library(dplyr)
d<-readRDS(file="TD4_mydata.rds")
d<-d%>%dplyr::select(demtherm,age,gender,edu,race)

In [ ]:
#Cadre 2
####
#
#Produisez des statistiques descriptives basiques sur ces variables (summary()).
summary(d)

In [ ]:
#Cadre 3
####
#Créez deux variables :
#1)white qui vaudra 1 si l'individu se définit comme blanc américain, 0 s'il se définit comme noir américain ou autre
#2)edu3 qui regroupera 1. et 2. en 'Pre-bac', 3. et 4. en 'Bac', 5 et 6. en 'Université', 8. et 9 en NA

#white
d$white<-ifelse(d$race==1,1,0) #1 si blanc, 0 sinon
d$white[d$race==9]<-NA  #attention à bien coder les variables manquantes en 'NA'
table(d$white, exclude=NULL) #on vérifie
d$white<-as.factor(d$white) #on définit cette variable comme facteur
levels(d$white)<-c("Non-White","White")

#edu3
summary(d$edu)
levels(d$edu)
d$edu3<-d$edu
levels(d$edu3)<-c("Pre-bac","Pre-bac","Bac","Bac","Université","Université",NA,NA)
levels(d$edu3)  #on vérifie
summary(d$edu3)

In [ ]:
#Cadre 4
####
#Effectuer la régression de *demtherm* sur *age*, *genre*, et les deux variables nouvellement crées: *edu3* et *white*. Commenter.
lm1<-lm(demtherm~gender+age+edu3+white, data=d)
summary(lm1)

In [ ]:
#Cadre 5
####
#Représenter graphiquement le score moyen des blancs américains et non-blancs (on prendra les hommes de niveau bac et d'âge de 40).
#catégorie pour laquelle on va prédire :
predict_for <- data.frame(expand.grid(gender = "Homme",
                                      age = 40, 
                                      edu3 = "Bac",
                                      white = c("Non-White","White")))
predict_for
#on prédit :
pred1<-predict(lm1,
               newdata=predict_for,
               se.fit=T,
               interval="confidence"
)
pred1
#on associe les deux dataframes:
pred1 <- data.frame(cbind(pred1$fit,predict_for))
pred1
#graphique :
library(ggplot2)
ggplot(pred1, aes(x=white,y=fit)) + geom_point() + geom_errorbar(aes(ymin=lwr,ymax=upr), width=0.1)  + xlab("") + ylab("Opinion pro-démocrate")

In [ ]:
#Cadre 6
####
#Refaire l'estimation en interagissant l'ethnie avec l'éducation :
lm2 <- lm(demtherm~gender+age+edu3+white+white:edu3, data=d)
summary(lm2)

In [ ]:
#Cadre 7
####
#OPTIONNEL: Représenter graphiquement les scores moyens des hommes de 40 ans selon leur niveau d'étude, pour les non-whites et whites (les scores moyens avec leu rintervalle de confiance en ordonnée, les niveaux d'éducation en absisse, et les origines ethniques en légende).
#catégories pour lesquelles on prédit:
predict_for<-data.frame(expand.grid(gender="Homme",
                                    age=40,
                                    white=c("Non-White","White"),
                                    edu3=c("Pre-bac","Bac","Université")))
predict_for

#valeurs prédites:
pred2<-predict(lm2,
               newdata=predict_for,
               se.fit=T,
               interval="confidence")
pred2

#combinaison des deux objets en 1 dataframe
pred2<-data.frame(cbind(pred2$fit,predict_for))
pred2

#on utilise ce dataframe pour créer le graphique:
ggplot(pred2, aes(x=edu3,y=fit,color=white)) +
  geom_point() +
  geom_errorbar(aes(ymin=lwr,ymax=upr),width=0.1) 